In [1]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar""")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "aulafia")
         .config("spark.hadoop.fs.s3a.secret.key", "aulafia@123")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

In [3]:
df_estado = spark.read.format('parquet').load('s3a://silver/dado_geografico/estado')
print('==========> df_estado.printSchema(): ', df_estado.printSchema())
print('==========> df_estado.count(): ', df_estado.count())
print('==========> df_estado.show(5, False): ', df_estado.show(5, False))

root
 |-- estado_id: integer (nullable = true)
 |-- estado_nome: string (nullable = true)
 |-- estado_sigla: string (nullable = true)
 |-- regiao_id: integer (nullable = true)
 |-- regiao_nome: string (nullable = true)
 |-- regiao_sigla: string (nullable = true)

==========> df_estado.printSchema():  None
==========> df_estado.count():  27
+---------+-----------+------------+---------+-----------+------------+
|estado_id|estado_nome|estado_sigla|regiao_id|regiao_nome|regiao_sigla|
+---------+-----------+------------+---------+-----------+------------+
|11       |Rondônia   |RO          |1        |Norte      |N           |
|12       |Acre       |AC          |1        |Norte      |N           |
|13       |Amazonas   |AM          |1        |Norte      |N           |
|14       |Roraima    |RR          |1        |Norte      |N           |
|15       |Pará       |PA          |1        |Norte      |N           |
+---------+-----------+------------+---------+-----------+------------+
only showi

In [4]:
df_municipio = spark.read.format('parquet').load('s3a://silver/dado_geografico/municipio')
print('==========> df_municipio.printSchema(): ', df_municipio.printSchema())
print('==========> df_municipio.count(): ', df_municipio.count())
print('==========> df_municipio.show(5, False): ', df_municipio.show(5, False))

root
 |-- municipio_id: long (nullable = true)
 |-- municipio_nome: string (nullable = true)
 |-- microregiao_id: long (nullable = true)
 |-- microregiao_nome: string (nullable = true)
 |-- microregiao_mesoregiao_id: long (nullable = true)
 |-- microregiao_mesoregiao_nome: string (nullable = true)
 |-- microregiao_mesoregiao_uf_id: integer (nullable = true)
 |-- microregiao_mesoregiao_uf_sigla: string (nullable = true)
 |-- microregiao_mesoregiao_uf_nome: string (nullable = true)
 |-- microregiao_mesoregiao_uf_regiao_id: integer (nullable = true)
 |-- microregiao_mesoregiao_uf_regiao_sigla: string (nullable = true)
 |-- microregiao_mesoregiao_uf_regiao_nome: string (nullable = true)
 |-- regiao_imediata_id: integer (nullable = true)
 |-- regiao_imediata_nome: string (nullable = true)
 |-- regiao_imediata_intermediaria_id: integer (nullable = true)
 |-- regiao_imediata_intermediaria_nome: string (nullable = true)
 |-- regiao_imediata_intermediaria_uf_id: integer (nullable = true)
 |-- r

In [5]:
df_fluxo_educ_superior = spark.read.format('parquet').load('s3a://bronze/inep/educacao_superior/indicadores_fluxo_educacao_superior')
print('==========> df_fluxo_educ_superior.printSchema(): ', df_fluxo_educ_superior.printSchema())
print('==========> df_fluxo_educ_superior.count(): ', df_fluxo_educ_superior.count())
print('==========> df_fluxo_educ_superior.show(5, False): ', df_fluxo_educ_superior.show(5, False))

root
 |-- CO_IES: long (nullable = true)
 |-- NO_IES: string (nullable = true)
 |-- TP_CATEGORIA_ADMINISTRATIVA: integer (nullable = true)
 |-- TP_ORGANIZACAO_ACADEMICA: integer (nullable = true)
 |-- CO_CURSO: long (nullable = true)
 |-- NO_CURSO: string (nullable = true)
 |-- CO_REGIAO: integer (nullable = true)
 |-- CO_UF: integer (nullable = true)
 |-- CO_MUNICIPIO: long (nullable = true)
 |-- TP_GRAU_ACADEMICO: integer (nullable = true)
 |-- TP_MODALIDADE_ENSINO: integer (nullable = true)
 |-- CO_CINE_ROTULO: string (nullable = true)
 |-- NO_CINE_ROTULO: string (nullable = true)
 |-- CO_CINE_AREA_GERAL: integer (nullable = true)
 |-- NO_CINE_AREA_GERAL: string (nullable = true)
 |-- NU_ANO_INGRESSO: integer (nullable = true)
 |-- NU_ANO_REFERENCIA: integer (nullable = true)
 |-- NU_PRAZO_INTEGRALIZACAO: integer (nullable = true)
 |-- NU_ANO_INTEGRALIZACAO: integer (nullable = true)
 |-- NU_PRAZO_ACOMPANHAMENTO: integer (nullable = true)
 |-- NU_ANO_MAXIMO_ACOMPANHAMENTO: integer (

In [6]:
df_fluxo_educ_superior_estmun = (df_fluxo_educ_superior.alias('educ')
     .join(df_estado.alias('est'), fn.col('educ.CO_UF') == fn.col('est.estado_id'), 'left')
     .join(df_municipio.alias('mun'), fn.col('educ.CO_MUNICIPIO') == fn.col('mun.municipio_id'), 'left')
     .select(fn.col('educ.CO_IES').alias('codigo_instituicao'),
             fn.col('educ.NO_IES').alias('nome_instituicao'),
             fn.col('educ.TP_CATEGORIA_ADMINISTRATIVA').alias('categoria_administrativa'),
             fn.col('educ.TP_ORGANIZACAO_ACADEMICA').alias('organizacao_academica'),
             fn.col('educ.CO_CURSO').alias('codigo_curso_graduacao'),
             fn.col('educ.NO_CURSO').alias('nome_curso_graduacao'),
             fn.col('educ.CO_REGIAO').alias('codigo_regiao_geografica_curso'),
             fn.col('est.regiao_nome').alias('nome_regiao_geografica_curso'), # df_estado.regiao_nome
             fn.col('educ.CO_UF').alias('codigo_unidade_federativa_curso'),
             fn.col('est.estado_nome').alias('nome_unidade_federativa_curso'), # df_estado.estado_nome
             fn.col('educ.CO_MUNICIPIO').alias('codigo_municipio_curso'),
             fn.col('mun.municipio_nome').alias('nome_municipio_curso'), # df_municipio.municipio_nome
             fn.col('educ.TP_GRAU_ACADEMICO').alias('tipo_grau_academico'),
             fn.col('educ.TP_MODALIDADE_ENSINO').alias('tipo_modalidade_ensino'),
             fn.col('educ.CO_CINE_ROTULO').alias('codigo_area_curso_classificacao_cine_brasil'),
             fn.col('educ.NO_CINE_ROTULO').alias('nome_area_curso_classificacao_cine_brasil'),
             fn.col('educ.CO_CINE_AREA_GERAL').alias('codigo_grande_area_curso_classificacao_cine_brasil'),
             fn.col('educ.NO_CINE_AREA_GERAL').alias('nome_grande_area_curso_classificacao_cine_brasil'),
             fn.col('educ.NU_ANO_INGRESSO').alias('ano_ingresso'),
             fn.col('educ.NU_ANO_REFERENCIA').alias('ano_referencia'),
             fn.col('educ.NU_PRAZO_INTEGRALIZACAO').alias('prazo_integralizacao_anos'),
             fn.col('educ.NU_ANO_INTEGRALIZACAO').alias('ano_integralizacao_curso'),
             fn.col('educ.NU_PRAZO_ACOMPANHAMENTO').alias('prazo_acompanhamento_curso_anos'),
             fn.col('educ.NU_ANO_MAXIMO_ACOMPANHAMENTO').alias('ano_maximo_acompanhamento_curso'),
             fn.col('educ.QT_INGRESSANTE').alias('quantidade_ingressante_curso'),
             fn.col('educ.QT_PERMANENCIA').alias('quantidade_permanencia_curso_ano_referencia'),
             fn.col('educ.QT_CONCLUINTE').alias('quantidade_concluinte_curso_ano_referencia'),
             fn.col('educ.QT_DESISTENCIA').alias('quantidade_desistencia_curso_ano_referencia'),
             fn.col('educ.QT_FALECIDO').alias('quantidade_falecimento_curso_ano_referencia'),
             fn.col('educ.TAP').alias('taxa_permanencia_tap'),
             fn.col('educ.TCA').alias('taxa_conclusao_acumulada_tca'), 
             fn.col('educ.TDA').alias('taxa_desistencia_acumulada_tda'),
             fn.col('educ.TCAN').alias('taxa_conclusso_anual_tcan'),
             fn.col('educ.TADA').alias('taxa_desistencia_anual_tada')
            )
)
print('==========> df_fluxo_educ_superior_estmun.printSchema(): ', df_fluxo_educ_superior_estmun.printSchema())
print('==========> df_fluxo_educ_superior_estmun.count(): ', df_fluxo_educ_superior_estmun.count())
print('==========> df_fluxo_educ_superior_estmun.show(5, False): ', df_fluxo_educ_superior_estmun.show(5, False))

root
 |-- codigo_instituicao: long (nullable = true)
 |-- nome_instituicao: string (nullable = true)
 |-- categoria_administrativa: integer (nullable = true)
 |-- organizacao_academica: integer (nullable = true)
 |-- codigo_curso_graduacao: long (nullable = true)
 |-- nome_curso_graduacao: string (nullable = true)
 |-- codigo_regiao_geografica_curso: integer (nullable = true)
 |-- nome_regiao_geografica_curso: string (nullable = true)
 |-- codigo_unidade_federativa_curso: integer (nullable = true)
 |-- nome_unidade_federativa_curso: string (nullable = true)
 |-- codigo_municipio_curso: long (nullable = true)
 |-- nome_municipio_curso: string (nullable = true)
 |-- tipo_grau_academico: integer (nullable = true)
 |-- tipo_modalidade_ensino: integer (nullable = true)
 |-- codigo_area_curso_classificacao_cine_brasil: string (nullable = true)
 |-- nome_area_curso_classificacao_cine_brasil: string (nullable = true)
 |-- codigo_grande_area_curso_classificacao_cine_brasil: integer (nullable = 

In [7]:
# (df_fluxo_educ_superior_estmun
#  .write
#  .format('parquet')
#  .mode('overwrite')
#  .save('s3a://silver/educacao_superior/indicadores_fluxo_educacao_superior')
#  )